![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Tool Selection Strategies: Improving Tool Choice

## Learning Objectives (25-30 minutes)
By the end of this notebook, you will understand:
1. **Common tool selection failures** and why they happen
2. **Strategies to improve tool selection** with clear naming and descriptions
3. **How LLMs select tools** and what influences their decisions
4. **Testing and debugging** tool selection issues
5. **Best practices** for tool organization and consolidation

## Prerequisites
- Completed `02_defining_tools.ipynb`
- Understanding of tool creation basics
- Redis Stack running with course data
- OpenAI API key configured

---

## Introduction

In this advanced notebook, you'll learn strategies to improve how LLMs select tools. When you have many tools, the LLM can get confused about which one to use. You'll learn techniques to make tool selection more reliable and accurate.

### What You'll Learn

- Common tool selection failures
- Strategies to improve tool selection
- Clear naming conventions
- Detailed descriptions with examples
- Testing and debugging tool selection

## Concepts: Tool Selection Challenges

### The Problem

As you add more tools, the LLM faces challenges:

**Scenario:** Imagine you're building a class agent with tools for searching, filtering, listing, finding, and browsing courses. A student asks "What computer science courses are available?" Which tool should the LLM use? Without clear guidance, it might pick the wrong one.

**With 3 tools:**
- ✅ Easy to choose
- ✅ Clear distinctions

**With 10+ tools:**
- ⚠️ Similar-sounding tools
- ⚠️ Overlapping functionality
- ⚠️ Ambiguous queries
- ⚠️ Wrong tool selection

### The Problem: Scale Matters

In our course agent, we might need tools for:
- Searching courses (by topic, department, difficulty, format)
- Getting course details (by code, by name)
- Checking prerequisites, enrollment, schedules
- Managing student records

**Quick math:** With 3-5 variations per category, you could easily have 15-20 tools. That's when tool selection becomes critical.

### Common Tool Selection Failures

**1. Similar Names**
```python
# Bad: Confusing names
get_course()      # Get one course? Or search for one?
get_courses()     # Get multiple? How many? Search or list all?
search_course()   # Search for one? Or many?
find_courses()    # Same as search_course()? Different how?
# The LLM asks the same questions you're asking now!
```

**2. Vague Descriptions**
```python
# Bad: Too vague
def search_courses():
    """Search for courses."""
    
# Good: Specific with examples
def search_courses():
    """Search for courses using semantic search.
    
    Use when students ask about:
    - Topics: 'machine learning courses'
    - Departments: 'computer science courses'
    - Characteristics: 'online courses' or 'easy courses'
    
    Returns: List of matching courses with relevance scores.
    """
```

**3. Overlapping Functionality**
```python
# Bad: Unclear when to use which tool
search_courses(query)           # Semantic search
filter_courses(department)      # Filter by department  
find_courses_by_topic(topic)    # Find by topic
# Problem: "computer science courses" could use ANY of these!

# Good: One tool with clear parameters
search_courses(
    query: str,                 # "computer science"
    department: str = None,     # Optional filter
    topic: str = None           # Optional filter
)
# Result: One clear entry point, no confusion
```

### How LLMs Select Tools

The LLM follows a decision process:

1. **Tool name** - First impression ("Does this sound relevant?")
2. **Tool description** - Main decision factor ("When should I use this?")
3. **Parameter descriptions** - Confirms choice ("Can I provide these parameters?")
4. **Context** - User's query and conversation ("Does this match the user's intent?")

**Think of it like this:** The LLM is reading a menu at a restaurant. Tool names are dish names, descriptions are the ingredients/explanation, and parameters are customization options. A vague menu leads to wrong orders!

**Key insight:** The LLM can't see your code, only the schema!

## Quick Check: Can You Spot the Problem?

Before we dive into code, look at these two tools:
```python
def get_course_info(code: str):
    """Get information about a course."""
    
def get_course_data(code: str):  
    """Get data for a course."""
```

**Question:** If a student asks "Tell me about CS101", which tool would you pick?

**Answer:** Impossible to tell! They sound identical. This is exactly what the LLM experiences with bad tool definitions. Let's fix this...

### What You'll Practice

In this notebook, we'll:

1. **Create confusing tools** with bad names and descriptions
2. **Test them** to see the LLM make wrong choices  
3. **Fix them** using the strategies above
4. **Test again** to verify improvements

You'll see actual tool selection failures and learn how to prevent them.

## Setup

In [ ]:
# Setup - Run this first
import os
import asyncio
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# LangChain imports
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Redis and course management
import redis
from redis_context_course.course_manager import CourseManager

load_dotenv()
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
redis_client = redis.from_url(REDIS_URL)
course_manager = CourseManager()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Setup complete - ready to test tool selection!")

## Demonstration: Bad Tool Selection

Let's create some confusing tools and see what happens when the LLM tries to choose between them.

In [ ]:
# Create confusing tools with bad names and descriptions

@tool
async def get_course(code: str) -> str:
    """Get a course."""
    try:
        course = await course_manager.get_course_by_code(code)
        if not course:
            return f"Course {code} not found."
        return f"{course.code}: {course.title}\n{course.description}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
async def get_courses(query: str) -> str:
    """Get courses."""
    try:
        results = await course_manager.search_courses(query, limit=3)
        if not results:
            return "No courses found."
        output = []
        for course in results:
            output.append(f"{course.code}: {course.title}")
        return "\n".join(output)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
async def search_course(topic: str) -> str:
    """Search course."""
    try:
        results = await course_manager.search_courses(topic, limit=5)
        if not results:
            return "No courses found."
        output = []
        for course in results:
            output.append(f"{course.code}: {course.title}")
        return "\n".join(output)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
async def find_courses(department: str) -> str:
    """Find courses."""
    try:
        results = await course_manager.search_courses(department, limit=5)
        if not results:
            return "No courses found."
        output = []
        for course in results:
            output.append(f"{course.code}: {course.title}")
        return "\n".join(output)
    except Exception as e:
        return f"Error: {str(e)}"

print("❌ Created 4 confusing tools with bad names and descriptions")

### Test the Confusion

Let's create an agent with these confusing tools and see what happens.

In [ ]:
# Create an agent with confusing tools
confusing_tools = [get_course, get_courses, search_course, find_courses]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful course advisor. Use the available tools to help students."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_functions_agent(llm, confusing_tools, prompt)
confusing_agent = AgentExecutor(agent=agent, tools=confusing_tools, verbose=True)

print("🤖 Created agent with confusing tools")

In [ ]:
# Test with ambiguous queries
test_queries = [
    "What computer science courses are available?",
    "Find me some programming courses",
    "Show me courses about databases"
]

print("🧪 Testing confusing tools with ambiguous queries...")
print("\nWatch which tools the LLM chooses and why!")

# Uncomment to test (will show verbose output)
# for query in test_queries:
#     print(f"\n{'='*50}")
#     print(f"Query: {query}")
#     print('='*50)
#     result = confusing_agent.invoke({"input": query})
#     print(f"Result: {result['output']}")

print("\n💡 Notice: The LLM might pick different tools for similar queries!")

## Improvement Strategies

Now let's fix the problems by applying the strategies we learned.

### Strategy 1: Clear, Specific Names

Replace vague names with specific, action-oriented names.

In [ ]:
# Strategy 1: Better names

@tool
async def get_course_details_by_code(course_code: str) -> str:
    """
    Get detailed information about a specific course using its course code.
    
    Use this when:
    - Student asks about a specific course code ("Tell me about CS101")
    - Student wants detailed course information
    - Student asks about prerequisites, credits, or full description
    
    Do NOT use for:
    - Searching for courses by topic (use search_courses_by_topic instead)
    - Finding multiple courses
    
    Returns: Complete course details including description, prerequisites, credits.
    """
    try:
        course = await course_manager.get_course_by_code(course_code.upper())
        if not course:
            return f"Course {course_code} not found. Please check the course code."
        
        details = f"**{course.code}: {course.title}**\n"
        details += f"Credits: {course.credits}\n"
        details += f"Description: {course.description}\n"
        if course.prerequisites:
            details += f"Prerequisites: {', '.join(course.prerequisites)}\n"
        return details
    except Exception as e:
        return f"Error getting course details: {str(e)}"

print("✅ Created tool with clear name and detailed description")

### Strategy 2: Detailed Descriptions with Examples

Add specific use cases and examples to guide the LLM.

In [ ]:
# Strategy 2: Rich descriptions with examples

@tool
async def search_courses_by_topic(query: str) -> str:
    """
    Search for courses using semantic similarity matching.
    
    Use this when students ask about:
    - Topics: 'machine learning courses', 'web development', 'databases'
    - Characteristics: 'beginner courses', 'online courses', 'project-based'
    - General exploration: 'what courses are available?', 'show me programming courses'
    - Department-related: 'computer science courses', 'math courses'
    
    Do NOT use for:
    - Specific course codes (use get_course_details_by_code instead)
    - Prerequisites checking (use check_prerequisites instead)
    
    Returns: List of up to 5 relevant courses with codes and titles, ranked by relevance.
    """
    try:
        results = await course_manager.search_courses(query, limit=5)
        if not results:
            return f"No courses found matching '{query}'. Try different keywords or broader terms."
        
        output = [f"Found {len(results)} courses matching '{query}':"]
        for i, course in enumerate(results, 1):
            output.append(f"{i}. {course.code}: {course.title}")
        return "\n".join(output)
    except Exception as e:
        return f"Error searching courses: {str(e)}"

print("✅ Created tool with rich description and clear examples")

### Strategy 3: Consolidate Overlapping Tools

Instead of multiple similar tools, create one flexible tool with clear parameters.

In [ ]:
# Strategy 3: Consolidated tool
# Instead of: get_course, get_courses, search_course, find_courses
# We now have: get_course_details_by_code + search_courses_by_topic

improved_tools = [get_course_details_by_code, search_courses_by_topic]

print("✅ Consolidated 4 confusing tools into 2 clear tools")
print("\nBefore: get_course, get_courses, search_course, find_courses")
print("After:  get_course_details_by_code, search_courses_by_topic")
print("\nResult: Clear distinction between getting ONE course vs SEARCHING for courses")

### Test the Improvements

Let's test the improved tools with the same queries.

In [ ]:
# Create agent with improved tools
improved_agent = create_openai_functions_agent(llm, improved_tools, prompt)
improved_executor = AgentExecutor(agent=improved_agent, tools=improved_tools, verbose=True)

print("🤖 Created agent with improved tools")
print("\n🧪 Test the same queries with improved tools:")

# Uncomment to test improvements
# for query in test_queries:
#     print(f"\n{'='*50}")
#     print(f"Query: {query}")
#     print('='*50)
#     result = improved_executor.invoke({"input": query})
#     print(f"Result: {result['output']}")

print("\n💡 Notice: More consistent tool selection with clear descriptions!")

## Key Takeaways

### What We Learned

1. **Tool selection problems scale quickly** - 3 tools are easy, 10+ tools create confusion
2. **Names matter** - Specific, action-oriented names beat generic ones
3. **Descriptions are critical** - Examples and use cases guide LLM decisions
4. **Consolidation helps** - Fewer, well-designed tools beat many similar ones
5. **Testing is essential** - Always verify tool selection with real queries

### Best Practices Summary

**✅ Do:**
- Use specific, descriptive tool names
- Include "Use this when..." examples in descriptions
- Specify what NOT to use the tool for
- Test with ambiguous queries
- Consolidate similar tools when possible

**❌ Don't:**
- Use vague names like `get_data` or `search`
- Write minimal descriptions like "Get courses"
- Create multiple tools that do similar things
- Assume the LLM will figure it out
- Skip testing with real queries

### Next Steps

Ready to practice these concepts? Continue with `03d_hands_on_tool_selection.ipynb` for guided exercises that will help you master tool selection optimization!